In [53]:
import pandas as pd
import igraph as ig
import networkx as nx

In [54]:
# Carregar o grafo usando igraph
g_igraph = ig.Graph.Read_GML('grafo_podado.gml')

# Converter o grafo igraph para networkx (caso precise)
G = nx.Graph(ig.Graph.to_networkx(g_igraph))

# Carregar o dataset com as votações polarizadas
votacoes_polarizadas = pd.read_csv('../data/votacoes_polarizadas.csv')

# Converter a coluna 'id_votacao' inteira para inteiros
votacoes_polarizadas['id_votacao'] = votacoes_polarizadas['id_votacao'].apply(lambda x: int(float(x)))

# Definir o ano que deseja filtrar (exemplo: 2024)
ano_filtro = 2022

# Filtrar o dataset "votacoes_polarizadas" pelo ano especificado
votacoes_polarizadas_filtrado = votacoes_polarizadas[votacoes_polarizadas['ano_votacao'] == ano_filtro]

# Carregar o dataset com os votos de cada parlamentar
votos_parlamentares = pd.read_csv('../data/dados abertos/votacao_parlamentar.csv')
votos_parlamentares = votos_parlamentares.dropna(subset=['id_deputado'])

# Converter a coluna 'id_votacao' inteira para inteiros
votos_parlamentares['id_deputado'] = votos_parlamentares['id_deputado'].apply(lambda x: int(float(x)))

In [55]:
# Função para identificar a comunidade de um deputado
def obter_comunidade(deputado_id):
    return G.nodes[deputado_id]['comunidade']

In [56]:
# Extrair o 'label' dos nós que pertencem à comunidade '0' (grupo direita) e converter para int
grupo_direita = [int(float(G.nodes[n]['label'])) for n in G.nodes if G.nodes[n]['comunidade'] == 0]
grupo_esquerda = [int(float(G.nodes[n]['label'])) for n in G.nodes if G.nodes[n]['comunidade'] == 1]

In [70]:
# Inicializar variáveis para contagem de sucessos de cada grupo
sucesso_direita = 0
sucesso_esquerda = 0
total_votacoes = 0

# Lista para armazenar os resultados detalhados
resultados_detalhados = []

# Iterar sobre cada votação para calcular as métricas
for _, votacao in votacoes_polarizadas_filtrado.iterrows():
    id_votacao = int(float(votacao['id_votacao']))
    aprovado = int(float(votacao['aprovacao']))  # 1 para aprovado, 0 para reprovado

    # Filtrar votos dos parlamentares dessa votação
    votos_na_votacao = votos_parlamentares[votos_parlamentares['id_votacao'] == id_votacao]

    # Contar votos de cada grupo
    votos_direita = votos_na_votacao[votos_na_votacao['id_deputado'].isin(grupo_direita)]
    votos_esquerda = votos_na_votacao[votos_na_votacao['id_deputado'].isin(grupo_esquerda)]

    # Verificar se há votos suficientes para cada grupo
    if votos_direita.empty or votos_esquerda.empty:
        continue

    # Contar número de votos de "Sim" e "Não"
    total_votos_sim = len(votos_na_votacao[votos_na_votacao['voto'] == 'Sim'])
    total_votos_nao = len(votos_na_votacao[votos_na_votacao['voto'] == 'Não'])

    # Calcular maioria de votos para cada grupo
    if not votos_direita['voto'].value_counts().empty:
        voto_majoritario_direita = votos_direita['voto'].value_counts().idxmax()
        total_votos_direita = len(votos_direita)
    else:
        voto_majoritario_direita = 'Indefinido'
        total_votos_direita = 0

    if not votos_esquerda['voto'].value_counts().empty:
        voto_majoritario_esquerda = votos_esquerda['voto'].value_counts().idxmax()
        total_votos_esquerda = len(votos_esquerda)
    else:
        voto_majoritario_esquerda = 'Indefinido'
        total_votos_esquerda = 0

    # Determinar qual grupo "ganhou" a votação
    if voto_majoritario_direita == 'Sim' and aprovado == 1:
        grupo_ganhador = 'Direita'
        sucesso_direita += 1
    elif voto_majoritario_direita == 'Não' and aprovado == 0:
        grupo_ganhador = 'Direita'
        sucesso_direita += 1
    elif voto_majoritario_esquerda == 'Sim' and aprovado == 1:
        grupo_ganhador = 'Esquerda'
        sucesso_esquerda += 1
    elif voto_majoritario_esquerda == 'Não' and aprovado == 0:
        grupo_ganhador = 'Esquerda'
        sucesso_esquerda += 1
    else:
        grupo_ganhador = 'Indeterminado'

    # Armazenar os detalhes da votação
    resultados_detalhados.append({
        'id_votacao': id_votacao,
        'aprovado': 'Sim' if aprovado == 1 else 'Não',
        'voto_direita': voto_majoritario_direita,
        'voto_esquerda': voto_majoritario_esquerda,
        'total_votos_sim': total_votos_sim,
        'total_votos_nao': total_votos_nao,
        'total_votos_direita': total_votos_direita,
        'total_votos_esquerda': total_votos_esquerda,
        'grupo_ganhador': grupo_ganhador
    })

    total_votacoes += 1

# Converter a lista de resultados detalhados para um DataFrame
df_resultados = pd.DataFrame(resultados_detalhados)

# Calcular a taxa de sucesso de cada grupo
taxa_sucesso_direita = (sucesso_direita / total_votacoes) * 100 if total_votacoes > 0 else 0
taxa_sucesso_esquerda = (sucesso_esquerda / total_votacoes) * 100 if total_votacoes > 0 else 0

In [71]:
# Exibir a tabela com os resultados
df_resultados

,id_votacao,aprovado,voto_direita,voto_esquerda,total_votos_sim,total_votos_nao,total_votos_direita,total_votos_esquerda,grupo_ganhador
0,2309053105,Sim,Sim,Não,206,153,40,31,Direita
1,2309053105,Sim,Sim,Não,206,153,40,31,Direita
2,232490052,Sim,Sim,Não,190,170,38,31,Direita
3,231912970,Não,Não,Sim,175,235,50,33,Direita
4,231130063,Não,Não,Sim,169,182,46,31,Direita
...,...,...,...,...,...,...,...,...,...
948,15460179,Sim,Sim,Não,234,175,47,33,Direita
949,15460165,Sim,Sim,Não,246,202,53,32,Direita
950,15460200,Não,Não,Sim,166,240,48,32,Direita
951,15460156,Não,Não,Sim,211,243,56,33,Direita


In [72]:
# Exibir as taxas de sucesso
print(f"\nTaxa de Sucesso Direita: {taxa_sucesso_direita:.2f}%")
print(f"Taxa de Sucesso Esquerda: {taxa_sucesso_esquerda:.2f}%")


Taxa de Sucesso Direita: 95.38%
Taxa de Sucesso Esquerda: 4.62%


In [74]:
# Inicializar dicionários para armazenar o número de vezes que cada parlamentar votou e foi leal ao seu grupo
lealdade_direita = {deputado: {'votos_totais': 0, 'votos_leais': 0, 'abstencoes': 0} for deputado in grupo_direita}
lealdade_esquerda = {deputado: {'votos_totais': 0, 'votos_leais': 0, 'abstencoes': 0} for deputado in grupo_esquerda}

# Iterar sobre cada votação
for _, votacao in votacoes_polarizadas_filtrado.iterrows():
    id_votacao = int(float(votacao['id_votacao']))

    # Filtrar votos dos parlamentares dessa votação
    votos_na_votacao = votos_parlamentares[votos_parlamentares['id_votacao'] == id_votacao]

    # Contar votos de cada grupo
    votos_direita = votos_na_votacao[votos_na_votacao['id_deputado'].isin(grupo_direita)]
    votos_esquerda = votos_na_votacao[votos_na_votacao['id_deputado'].isin(grupo_esquerda)]

    # Verificar se há votos suficientes para cada grupo
    if votos_direita.empty or votos_esquerda.empty:
        continue

    # Calcular maioria de votos para cada grupo
    if not votos_direita['voto'].value_counts().empty:
        voto_majoritario_direita = votos_direita['voto'].value_counts().idxmax()
    else:
        voto_majoritario_direita = 'Indefinido'

    if not votos_esquerda['voto'].value_counts().empty:
        voto_majoritario_esquerda = votos_esquerda['voto'].value_counts().idxmax()
    else:
        voto_majoritario_esquerda = 'Indefinido'

    # Verificar a lealdade dos membros da Direita
    deputados_direita_que_votaram = set(votos_direita['id_deputado'])
    for deputado in grupo_direita:
        if deputado in deputados_direita_que_votaram:
            lealdade_direita[deputado]['votos_totais'] += 1
            voto_deputado = votos_direita[votos_direita['id_deputado'] == deputado]['voto'].values[0]
            if voto_deputado == voto_majoritario_direita:
                lealdade_direita[deputado]['votos_leais'] += 1
        else:
            lealdade_direita[deputado]['abstencoes'] += 1

    # Verificar a lealdade dos membros da Esquerda
    deputados_esquerda_que_votaram = set(votos_esquerda['id_deputado'])
    for deputado in grupo_esquerda:
        if deputado in deputados_esquerda_que_votaram:
            lealdade_esquerda[deputado]['votos_totais'] += 1
            voto_deputado = votos_esquerda[votos_esquerda['id_deputado'] == deputado]['voto'].values[0]
            if voto_deputado == voto_majoritario_esquerda:
                lealdade_esquerda[deputado]['votos_leais'] += 1
        else:
            lealdade_esquerda[deputado]['abstencoes'] += 1

# Calcular a lealdade percentual e taxa de abstenção de cada membro da Direita e Esquerda
lealdade_direita_percentual = {deputado: (dados['votos_leais'] / dados['votos_totais']) * 100 if dados['votos_totais'] > 0 else 0 for deputado, dados in lealdade_direita.items()}
lealdade_esquerda_percentual = {deputado: (dados['votos_leais'] / dados['votos_totais']) * 100 if dados['votos_totais'] > 0 else 0 for deputado, dados in lealdade_esquerda.items()}

abstencao_direita_percentual = {deputado: (dados['abstencoes'] / (dados['votos_totais'] + dados['abstencoes'])) * 100 if (dados['votos_totais'] + dados['abstencoes']) > 0 else 0 for deputado, dados in lealdade_direita.items()}
abstencao_esquerda_percentual = {deputado: (dados['abstencoes'] / (dados['votos_totais'] + dados['abstencoes'])) * 100 if (dados['votos_totais'] + dados['abstencoes']) > 0 else 0 for deputado, dados in lealdade_esquerda.items()}

# Calcular a lealdade média e abstenção média de cada grupo
lealdade_media_direita = sum(lealdade_direita_percentual.values()) / len(lealdade_direita_percentual)
lealdade_media_esquerda = sum(lealdade_esquerda_percentual.values()) / len(lealdade_esquerda_percentual)

abstencao_media_direita = sum(abstencao_direita_percentual.values()) / len(abstencao_direita_percentual)
abstencao_media_esquerda = sum(abstencao_esquerda_percentual.values()) / len(abstencao_esquerda_percentual)

# Exibir os resultados de lealdade e abstenção de cada membro da Direita
print("Lealdade e Abstenção dos Membros da Direita:")
for deputado, lealdade in lealdade_direita_percentual.items():
    abstencao = abstencao_direita_percentual[deputado]
    print(f"Deputado {deputado}: {lealdade:.2f}% de lealdade, {abstencao:.2f}% de abstenção")

# Exibir os resultados de lealdade e abstenção de cada membro da Esquerda
print("\nLealdade e Abstenção dos Membros da Esquerda:")
for deputado, lealdade in lealdade_esquerda_percentual.items():
    abstencao = abstencao_esquerda_percentual[deputado]
    print(f"Deputado {deputado}: {lealdade:.2f}% de lealdade, {abstencao:.2f}% de abstenção")

# Exibir a lealdade média e abstenção média de cada grupo
print(f"\nLealdade Média da Direita: {lealdade_media_direita:.2f}%")
print(f"Lealdade Média da Esquerda: {lealdade_media_esquerda:.2f}%")
print(f"Abstenção Média da Direita: {abstencao_media_direita:.2f}%")
print(f"Abstenção Média da Esquerda: {abstencao_media_esquerda:.2f}%")

Lealdade e Abstenção dos Membros da Direita:
Deputado 178901: 100.00% de lealdade, 54.56% de abstenção
Deputado 74459: 85.63% de lealdade, 10.91% de abstenção
Deputado 74655: 55.22% de lealdade, 2.52% de abstenção
Deputado 204372: 47.75% de lealdade, 4.41% de abstenção
Deputado 204358: 75.15% de lealdade, 12.59% de abstenção
Deputado 178916: 59.22% de lealdade, 16.89% de abstenção
Deputado 160599: 94.93% de lealdade, 11.02% de abstenção
Deputado 204562: 80.95% de lealdade, 22.35% de abstenção
Deputado 204485: 81.76% de lealdade, 15.42% de abstenção
Deputado 178996: 72.20% de lealdade, 16.58% de abstenção
Deputado 160673: 88.40% de lealdade, 11.33% de abstenção
Deputado 204575: 50.19% de lealdade, 16.58% de abstenção
Deputado 204461: 72.31% de lealdade, 4.51% de abstenção
Deputado 204452: 86.17% de lealdade, 4.41% de abstenção
Deputado 160653: 70.69% de lealdade, 14.80% de abstenção
Deputado 178910: 55.93% de lealdade, 0.00% de abstenção
Deputado 204377: 90.65% de lealdade, 15.84% de ab